In [3]:
import pandas as pd

In [4]:
ticker = "NFLX"

In [5]:
# same as previous, just use the pct_change function, and the to_frame
def read_annual_data(csv_file, ticker):
    data = pd.read_csv(csv_file, index_col=0)
    data = data[ticker]
    data = data.to_frame()
    # add column called percent change, to keep original data
    # detail in number we want to look at
    data['%-chg'] = data.pct_change()
    data = data.loc[2009:2019]
    return data

In [6]:
# same as previous, just use the pct_change function, and the to_frame
def read_quarterly_data(csv_file, ticker):
    data = pd.read_csv(csv_file, index_col=0, parse_dates=True)
    data = data[ticker]
    data = data.resample('Y', convention='end').agg('mean')
    data = data.to_frame()
    # add column called percent change, to keep original data
    # detail in number we want to look at
    data['%-chg'] = data[ticker].pct_change()
    data = data.loc['2009-01-01':'2019-12-31']
    # same as before
    data.index = data.index.strftime("%Y")
    return data

In [7]:
eps = read_annual_data("50-eps.csv", ticker)
fcf = read_annual_data("50-fcf.csv", ticker)
rev = read_annual_data("50-rev.csv", ticker)
bvps = read_quarterly_data("50-bvps.csv", ticker)

In [8]:
def add_annual_data_2col(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Create a new chart object.
    chart = workbook.add_chart({'type': 'line'})

    # Add a series to the chart.
    chart.add_series({
        'name': sheet_name,
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 1, 11, 1],
    })
    
    # Create a new column chart.
    column_chart = workbook.add_chart({'type': 'column'})
    
    # Add a series to the chart.
    column_chart.add_series({
        'name': '%-chg',
        # categories and values same just values is column 2 instead of 
        # 1
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 2, 11, 2],
        #use opposite y axis becuase one is percentage and 
        # one is actual values
        'y2_axis': True,
    })
    # combine 2nd y axis with original, column chart with chart
    chart.combine(column_chart)
    chart.set_title({'name': sheet_name})
    chart.set_x_axis({'name': 'Date'})
    chart.set_y_axis({'name': 'Value'})
    column_chart.set_y2_axis({'name': '%-chg'})
    
    #to set the labels on x axis not on 0 and rotate by 45
    chart.set_x_axis({
        'label_position': 'low',
        'num_font': {'rotation': 45}
    })
    
    #Insert the chart into the worksheet.
    worksheet.insert_chart('D1', chart)

In [9]:
writer = pd.ExcelWriter('data4.xlsx', engine='xlsxwriter')

add_annual_data_2col(writer, rev, 'Revenue')
add_annual_data_2col(writer, eps, 'EPS')
add_annual_data_2col(writer, fcf, 'FCF')
add_annual_data_2col(writer, bvps, 'BVPS')

writer.close()

It shows in FCF that Netflix has no FREE cash flow

A company with negative free cash flow indicates an inability to generate enough cash to support the business. Free cash flow tracks the cash a company has left over after meeting its operating expenses.